In [93]:
require 'nyaplot'

false

In [4]:
require 'daru'

true

In [5]:
N = 10
M = [0, 1, 3, 9]

def normal_rand(mu = 0, sigma = 1.0)
  random = Random.new
  (Math.sqrt(-2 * Math.log(random.rand)) * Math.sin(2 * Math::PI * random.rand) * sigma) + mu
end


def create_dataset(num)
  dataset = Daru::DataFrame.new({'x': [], 'y': []})  

  num.times do |i|
    x = i.to_f / (num - 1).to_f
    y = Math.sin(2 * Math::PI * x) + normal_rand(0, 0.3)
    dataset.add_row(Daru::Vector.new([x, y], index: [:x, :y]))
  end
  
  return dataset
end

def rms_error(dataset, f)
  err = 0.0
  dataset.each_row do |line|
    err += 0.5 * (line.y - f.call(line.x))**2
  end

  Math.sqrt(2 * err / dataset.size)
end

def resolve(dataset, m)
  t = dataset.y

  columns = {}
  (m+1).times do |i|
    columns["x**#{i}"] = dataset.x ** i
  end
  phi = Daru::DataFrame.new(columns)

  tmp = (phi.transpose.to_matrix * phi.to_matrix).inv
  ws = (tmp * phi.transpose.to_matrix) * Vector.elements(t.to_a)

  f = lambda {|x|
    y = 0
    ws.each_with_index do |w, i|
      y += w * (x ** i)
    end

    y
  }

  return f, ws
end


:resolve

In [94]:
train_set = create_dataset(N)
test_set = create_dataset(N)
df_ws = {}

fig = Nyaplot::Frame.new

M.each_with_index do |m, c|
  f, ws = resolve(train_set, m)
  df_ws["M=#{m}"] = Daru::Vector.new(ws, name: "M=#{m}")
  
  plot = Nyaplot::Plot.new
  sc = plot.add_with_df(train_set.to_nyaplotdf, :scatter, :x, :y)
  sc.title("train_set")
  sc.color('blue')
  
  linex = (0..1).step(1.0 / (101 - 1)).to_a
  liney = linex.map do |x|
    Math.sin(2 * Math::PI * x)
  end
  line_answer = plot.add(:line, linex, liney)
  line_answer.title('answer')
  line_answer.color('green')
  
  liney = linex.map do |x|
    f.call(x)
  end
  line_erms = plot.add(:line, linex, liney)
  line_erms.title("E(RMS#{sprintf("%.2f", rms_error(train_set, f))})")
  line_erms.color('red')

  plot.configure do
    x_label("M=#{m}")
    y_label('')
    xrange([-0.05, 1.05])
    yrange([-1.5, 1.5])
    legend(true)
    height(300)
    width(490)
  end
  
  fig.add(plot)
end

puts 'Table ot the coefficients'
puts Daru::DataFrame.new(df_ws).inspect

fig.show

df = Daru::DataFrame.new({m: [], 'Training set': [], 'Test set': []})  
10.times do |m|
  f, ws = resolve(train_set, m)
  train_error = rms_error(train_set, f)
  test_error = rms_error(test_set, f)
  df.add_row(Daru::Vector.new([m, train_error, test_error], index: [:m, 'Training set'.to_sym, 'Test set'.to_sym]))
end

df.plot(type: [:line, :line], x: [:m, :m], y: ['Training set'.to_sym, 'Test set'.to_sym]) do |plot, diagrams|
  train_set_diagram = diagrams[0]
  test_set_diagram = diagrams[1]
  
  train_set_diagram.title('Training set')
  train_set_diagram.color('blue')
  test_set_diagram.title('Test set')
  test_set_diagram.color('green')
  
  plot.x_label('M')
  plot.y_label('RMS Error')
  plot.yrange([0, 0.9])
  plot.legend(true)
end

Table ot the coefficients
#<Daru::DataFrame(10x4)>
                   M=0        M=1        M=3        M=9
          0 -0.0952916 0.78390541 0.01408700 -0.0371439
          1        nil -1.7583941 10.6828211 -0.2262255
          2        nil        nil -32.524617 380.770623
          3        nil        nil 21.5750079 -5029.5633
          4        nil        nil        nil 29300.3051
          5        nil        nil        nil -92767.194
          6        nil        nil        nil 169482.046
          7        nil        nil        nil -178388.81
          8        nil        nil        nil 100474.672
          9        nil        nil        nil -23452.379


#<Nyaplot::Frame:0x007f490c8f7148 @properties={:panes=>[#<Nyaplot::Plot:0x007f490c8fcd00 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f490a48d0f0 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :title=>"train_set", :color=>"blue"}, :data=>"c8e75c10-7c6c-4762-ba93-2489dad60f8d"}, @xrange=[0.0, 1.0], @yrange=[-1.192702397107344, 0.9296143863243069]>, #<Nyaplot::Diagram:0x007f490c91aff8 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"answer", :color=>"green"}, :data=>"05ea060a-96fa-44b9-8464-6482f9b31e3f"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007f490a468e08 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"E(RMS0.78)", :color=>"red"}, :data=>"d71fc5c9-5984-4f8e-a576-2b587bb76307"}, @xrange=[0.0, 1.0], @yrange=[-0.09529163915867203, -0.09529163915867203]>], :options=>{:x_label=>"M=0", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-1.5, 1.5], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f490c94ecb8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f490c954348 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :title=>"train_set", :color=>"blue"}, :data=>"d6855721-1ddb-4400-b4ce-fb234b1b7df6"}, @xrange=[0.0, 1.0], @yrange=[-1.192702397107344, 0.9296143863243069]>, #<Nyaplot::Diagram:0x007f490c9659b8 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"answer", :color=>"green"}, :data=>"6b073879-562b-4215-8314-933eb5ed4a23"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007f490c985b50 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"E(RMS0.55)", :color=>"red"}, :data=>"88a769e8-a376-487d-96a5-439900c57abd"}, @xrange=[0.0, 1.0], @yrange=[-0.9744886964913466, 0.7839054181740024]>], :options=>{:x_label=>"M=1", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-1.5, 1.5], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f490c9a5428 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f4909f1c2d8 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :title=>"train_set", :color=>"blue"}, :data=>"bf161822-5337-4758-977e-80a98e3409a9"}, @xrange=[0.0, 1.0], @yrange=[-1.192702397107344, 0.9296143863243069]>, #<Nyaplot::Diagram:0x007f490c9c1970 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"answer", :color=>"green"}, :data=>"8429ecc8-02d6-4547-9709-9eb03c1e6429"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007f490c9e18d8 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"E(RMS0.17)", :color=>"red"}, :data=>"6ce4528b-8092-47b3-b6a7-99bd76760473"}, @xrange=[0.0, 1.0], @yrange=[-1.209007354469355, 1.0229499471976364]>], :options=>{:x_label=>"M=3", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-1.5, 1.5], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f490ca16650 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f490ca1c208 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :title=>"train_set", :color=>"blue"}, :data=>"5f5e1a3d-5613-4483-a932-fcbcc47e3bda"}, @xrange=[0.0, 1.0], @yrange=[-1.192702397107344, 0.9296143863243069]>, #<Nyaplot::Diagram:0x007f490ca33110 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"answer", :color=>"green"}, :data=>"12e3ddae-9abf-4285-9e43-ab5620aa96af"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007f4909f65118 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"E(RMS0.00)", :color=>"red"}, :data=>"36759630-2ac3-4f3e-9ec9-8d89d66a900b"}, @xrange=[0.0, 1.0], @yrange=[-1.2337707564483935, 0.9623484355740396]>], :options=>{:x_label=>"M=9", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-1.5, 1.5], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>], :data=>{"c8e75c10-7c6c-4762-ba93-2489dad60f8d"=>#<Nyaplot::DataFrame:0x007f490a48e540 @name="c8e75c10-7c6c-4762-ba93-2489dad60f8d", @rows=[{:x=>0.0, :y=>-0.03714447948592249}, {:x=>0.11

#<Nyaplot::Frame:0x007f490c4ca070 @properties={:panes=>[#<Nyaplot::Plot:0x007f490c4f8a10 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f490c4caf70 @properties={:type=>:line, :options=>{:x=>:m, :y=>:"Training set", :title=>"Training set", :color=>"blue"}, :data=>"b6143e90-61b0-418d-b6b1-c1cf8d73d6cf"}, @xrange=[0, 9], @yrange=[0.00012369142490714662, 0.7831136885488286]>, #<Nyaplot::Diagram:0x007f490c4ca930 @properties={:type=>:line, :options=>{:x=>:m, :y=>:"Test set", :title=>"Test set", :color=>"green"}, :data=>"b6143e90-61b0-418d-b6b1-c1cf8d73d6cf"}, @xrange=[0, 9], @yrange=[0.2672359247299703, 0.6756179442806174]>], :options=>{:x_label=>"M", :y_label=>"RMS Error", :yrange=>[0, 0.9], :legend=>true, :zoom=>true, :width=>800, :xrange=>[0, 9]}}>], :data=>{"b6143e90-61b0-418d-b6b1-c1cf8d73d6cf"=>#<Nyaplot::DataFrame:0x007f490c4cb768 @name="b6143e90-61b0-418d-b6b1-c1cf8d73d6cf", @rows=[{:"Test set"=>0.6756179442806174, :"Training set"=>0.7831136885488286, :m=>0}, {:"Test set"=>0.6186019214101631, :"Training set"=>0.5462107925196034, :m=>1}, {:"Test set"=>0.6192433068496007, :"Training set"=>0.5460171736913962, :m=>2}, {:"Test set"=>0.30760179327450854, :"Training set"=>0.16610698582066982, :m=>3}, {:"Test set"=>0.2672359247299703, :"Training set"=>0.14420554054595217, :m=>4}, {:"Test set"=>0.27985427263061685, :"Training set"=>0.10186538469144481, :m=>5}, {:"Test set"=>0.2799297482315192, :"Training set"=>0.08508719488715642, :m=>6}, {:"Test set"=>0.2857484187926056, :"Training set"=>0.06430481695198287, :m=>7}, {:"Test set"=>0.2774339365373484, :"Training set"=>0.03149288980596075, :m=>8}, {:"Test set"=>0.27330638129667634, :"Training set"=>0.00012369142490714662, :m=>9}]>}, :extension=>[]}>